<a href="https://colab.research.google.com/github/bckim9489/agent_practice/blob/main/agent_pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Common Setup

참고 : 아래 Secret으로 Key 등록해서 실습함
1.   LLM API Key (GPT, claude 등) : OPENAI-KEY 로 등록했음
2.   Tavily Serach API Key (https://app.tavily.com/home) : TAVIL-KEY 로 등록했음



In [1]:
!pip -q install -U langgraph langchain-openai langchain-community wikipedia numexpr tavily-python httpx==0.27.2

In [2]:
import os
from langchain_openai import ChatOpenAI
from google.colab import userdata

# LLM API Key
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI-KEY")

# Tools
# Search API Key
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY-KEY")

llm = ChatOpenAI(model="gpt-4o", temperature=0)


## ㄴ 0.1 Tools

In [3]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_core.tools import tool
import numexpr

# Wikipedia tool
wiki = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=2000)
)

# 기존 wiki wrapper 재사용
wiki_lookup_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=4000)

@tool
def docstore_lookup(title: str) -> str:
    """
    Lookup a specific document by title from the docstore (Wikipedia).
    Input should be a precise title or entity name.
    """
    return wiki_lookup_wrapper.run(title)

# Tavily Search tool
search_tool = TavilySearchResults(max_results=5)

# Calculator tool (llm-math 대체)
@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression.
    Input must be a valid expression like '345*872' or '12/4+9'.
    """
    try:
        return str(numexpr.evaluate(expression))
    except Exception as e:
        return f"Error: {e}"


tools = [wiki, calculator, search_tool, docstore_lookup]

/tmp/ipython-input-4054356365.py:25: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=5)


# 1. Zero-Shot ReAct
*   기억 없음(Stateless)
*   **LLM + Tools**





## ㄴ 1.1 Description

### ㄴ 1.1.1 Workflow


```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
LLM: 질문 해석 (Thought / Reasoning)
  ↓
Tool 필요 여부 판단
  ├─ 필요 없음 → 바로 답 생성
  └─ 필요 있음 → Tool 선택 (Action)
                      ↓
                Tool 실행 (Python / API / DB 등)
                      ↓
                결과 반환 (Observation)
                      ↓
LLM이 결과 반영하여 다시 Reasoning
  ↓
(필요 시 Thought → Action → Observation 반복)
  ↓
Final Answer 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시

```

### ㄴ 1.1.2 Inner ReAct Loop



``` javascript
while 문제 해결 전:
    Thought      → 어떻게 풀지 스스로 판단
    Action       → 사용할 Tool 선택
    Observation  → Tool 실행 결과 받기
    Thought      → 결과 보고 다음 행동 결정
```



## ㄴ 1.2 Code

In [4]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

/tmp/ipython-input-2357398768.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


In [5]:
q = """위키피디아에서 찾기 좋은 '검색어(제목형 키워드)'를 3개 만들어서,
그 키워드로 Wikipedia tool을 사용해 검색하고 근거를 요약해줘.
주제: Pinus densiflora 옮겨심기(이식) 적기"""

In [6]:
result = agent.invoke({"messages": [("user", q)]})

print(result["messages"][-1].content)

위키피디아에서 "Pinus densiflora transplanting season", "best time to transplant Japanese red pine", "Pinus densiflora cultivation and care"라는 검색어로 검색을 시도했으나, 관련된 유의미한 결과를 찾을 수 없었습니다. 

이 주제에 대한 정보를 얻기 위해서는 다른 방법을 사용하거나, 보다 구체적인 자료를 찾아보는 것이 필요할 수 있습니다. 예를 들어, 원예 관련 서적이나 전문 웹사이트에서 정보를 찾는 것이 도움이 될 수 있습니다.


## ㄴ 1.3 Verbose

In [7]:
from langchain_core.messages import AIMessage, ToolMessage

for step in agent.stream(
    {"messages": [("user", q)]},
    stream_mode="values",
):
    msg = step["messages"][-1]

    # LLM이 생각/결정한 내용
    if isinstance(msg, AIMessage):
        print("\n AI MESSAGE")
        print(msg.content)

        if getattr(msg, "tool_calls", None):
            print(" TOOL CALL:", msg.tool_calls)

    # Tool 실행 결과
    elif isinstance(msg, ToolMessage):
        print("\n TOOL RESULT")
        print("tool:", msg.name)
        print(msg.content[:1000])  # 너무 길면 앞부분만



 AI MESSAGE

 TOOL CALL: [{'name': 'wikipedia', 'args': {'query': 'Pinus densiflora transplanting season'}, 'id': 'call_nA2Nz5ZCaBsJ690QlYhHQnrH', 'type': 'tool_call'}, {'name': 'wikipedia', 'args': {'query': 'best time to transplant Japanese red pine'}, 'id': 'call_JrITnlpgs5AYptyvmBRdvTjz', 'type': 'tool_call'}, {'name': 'wikipedia', 'args': {'query': 'Pinus densiflora cultivation and care'}, 'id': 'call_SGoTvyZyVMRqo6HKN4kFu6UZ', 'type': 'tool_call'}]

 TOOL RESULT
tool: wikipedia
No good Wikipedia Search Result was found

 AI MESSAGE
위키피디아에서 "Pinus densiflora transplanting season", "best time to transplant Japanese red pine", "Pinus densiflora cultivation and care"라는 검색어로 검색을 시도했으나, 관련된 유의미한 결과를 찾을 수 없었습니다. 

이 주제에 대한 정보를 얻기 위해서는 다른 방법을 사용하거나, 보다 구체적인 자료를 찾아보는 것이 필요할 수 있습니다. 예를 들어, 원예 관련 서적이나 전문 웹사이트에서 정보를 찾는 것이 도움이 될 수 있습니다.


# 2. Conversational ReAct
*   메모리 사용
*   **LLM + Tools + Checkpointer**



## ㄴ 2.1 Description

### ㄴ 2.1.1 Workflow

```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
이전 대화 기록(Memory / State) 로드
  ↓
LLM: 현재 질문 + 대화 히스토리 함께 해석
  ↓
사용자의 의도 파악 (맥락 기반 Reasoning)
  ↓
Tool 필요 여부 판단
  ├─ 필요 없음 → 바로 답 생성
  └─ 필요 있음 → Tool 선택 (Action)
                      ↓
                Tool 실행 (Python / API / DB 등)
                      ↓
                결과 반환 (Observation)
                      ↓
LLM이 결과 + 이전 대화 맥락을 반영하여 다시 Reasoning
  ↓
(필요 시 Thought → Action → Observation 반복)
  ↓
Final Answer 생성
  ↓
대화 Memory(State)에 이번 메시지 저장
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시
```

### ㄴ 2.1.2 Inner ReAct Loop

```javascript
while 문제 해결 전:
    Load Memory        → 이전 대화 불러오기
    Thought            → 맥락 기반으로 판단
    Action             → Tool 필요하면 실행
    Observation        → Tool 결과 받기
    Update Memory      → 새 정보 저장
```

## ㄴ 2.2 Code

In [8]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()
agent = create_react_agent(llm, tools, checkpointer=checkpointer)

/tmp/ipython-input-1647517424.py:5: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools, checkpointer=checkpointer)


In [9]:
config = {"configurable": {"thread_id": "chat-001"}}
q1 = "내 출생년도는 1994년이야. 지금은 계산하지 말고 이 정보만 기억해."
q2 = "그럼 내가 2026년에 몇 살인지 계산기를 꼭 사용해서 나이를 계산해서 알려줘"
q3 = "올해는 무슨 해이며 나는 무슨 띠인지 알려줘"

In [10]:
result = agent.invoke({"messages": [("user", q1)]}, config=config)
print(result["messages"][-1].content)
result = agent.invoke({"messages": [("user", q2)]}, config=config)
print(result["messages"][-1].content)
result = agent.invoke({"messages": [("user", q3)]}, config=config)
print(result["messages"][-1].content)


알겠습니다! 당신의 출생년도는 1994년입니다. 이 정보를 기억하겠습니다.
2026년에 당신은 32살이 됩니다.
2023년은 "계묘년"으로, 토끼의 해입니다. 그리고 당신이 태어난 1994년은 "갑술년"으로, 개의 띠입니다.


## ㄴ 2.3 Verbose

### ㄴ 2.3.1 Test Setup

In [15]:
from langchain_core.messages import AIMessage, ToolMessage
import time
from openai import RateLimitError

from langchain_core.messages import AIMessage, ToolMessage

def stream_with_retry(agent, payload, config, stream_mode="values", max_retries=6):
    """
    agent.stream(...) 를 RateLimitError(429) 발생 시 backoff 재시도.
    """
    attempt = 0
    while True:
        try:
            for step in agent.stream(payload, config=config, stream_mode=stream_mode):
                yield step
            return  # 정상 종료
        except RateLimitError as e:
            attempt += 1
            if attempt > max_retries:
                raise

            # 간단 backoff (점점 더 기다림)
            wait = min(20, 1.5 * attempt)
            print(f"\n RateLimit(429). retry in {wait:.1f}s ...")
            time.sleep(wait)



def run_verbose(question: str, label: str, thread_id: str):
    print("\n" + "="*20 + f" {label} (thread_id={thread_id}) " + "="*20)
    print("USER:", question)

    cfg = {"configurable": {"thread_id": thread_id}}

    payload = {"messages": [("user", question)]}

    last_ai = None
    for step in stream_with_retry(agent, payload, cfg, stream_mode="values"):
        msg = step["messages"][-1]

        if isinstance(msg, AIMessage):
            last_ai = msg
            if msg.content:
                print("\nAI:", msg.content)
            if getattr(msg, "tool_calls", None):
                print("tool_calls:", msg.tool_calls)

        elif isinstance(msg, ToolMessage):
            print("\nTOOL RESULT:", msg.name)
            print(msg.content[:800])

    return last_ai.content if last_ai else None

#메모리 사용하는지 A/B 테스트
def ab_test(q1: str, q2: str, q3:str, thread_a: str="mem-A", thread_b: str="mem-B", thread_c: str="mem-C"):
    print("\n\n" + "#"*10 + " A/B MEMORY TEST START " + "#"*10)

    # A: q1 using thread_id a
    run_verbose(q1, "A-1: seed memory (q1)", thread_a)


    # B: q2 using thread_id b
    run_verbose(q2, "B-1: q2 only (no memory)", thread_b)


    # B: q3 using thread_id c
    run_verbose(q3, "C-1: q3 only (no memory)", thread_c)

    print("\n" + "#"*10 + " A/B MEMORY TEST END " + "#"*10)


### ㄴ 2.3.2 Test A (같은 thread_id)

*   Q1을 thread_id A 에 저장
*   Q2를 thread_id A 에 질의
----------------------------
| Q2에서 메모리에 저장한 Q1을 활용하여 답변할 것으로 예상

In [16]:
# 같은 thread_id
run_verbose(q1, "Q1", "chat-001")
run_verbose(q2, "Q2", "chat-001")
run_verbose(q3, "Q3", "chat-001")


==================== Q1 (thread_id=chat-001) ====================
USER: 내 출생년도는 1994년이야. 지금은 계산하지 말고 이 정보만 기억해.

AI: 알겠습니다! 당신의 출생년도는 1994년입니다. 이 정보를 기억하겠습니다.

==================== Q2 (thread_id=chat-001) ====================
USER: 그럼 내가 2026년에 몇 살인지 계산기를 꼭 사용해서 나이를 계산해서 알려줘
tool_calls: [{'name': 'calculator', 'args': {'expression': '2026-1994'}, 'id': 'call_EjyEA8vW8d8tGm6zRh40VJEy', 'type': 'tool_call'}]

TOOL RESULT: calculator
32

AI: 2026년에 당신은 32살이 됩니다.

==================== Q3 (thread_id=chat-001) ====================
USER: 올해는 무슨 해이며 나는 무슨 띠인지 알려줘

 RateLimit(429). retry in 1.5s ...

 RateLimit(429). retry in 3.0s ...
tool_calls: [{'name': 'tavily_search_results_json', 'args': {'query': '2023년은 무슨 해'}, 'id': 'call_mGd5WnSNbmfm9kF48YkLh7f1', 'type': 'tool_call'}, {'name': 'tavily_search_results_json', 'args': {'query': '1994년은 무슨 띠'}, 'id': 'call_Pot4oxzGOHM2SrljvgZdCoz7', 'type': 'tool_call'}]

TOOL RESULT: tavily_search_results_json
[{"title": "1994 - Wikipedia", "url": "http

'2023년은 "계묘년"으로, 토끼의 해입니다. 그리고 당신이 태어난 1994년은 "갑술년"으로, 개의 띠입니다.'

### ㄴ 2.3.3 TEST B (다른 thread_id)

*   Q1을 thread_id A 에 저장
*   Q2를 thread_id B에 질의
----------------------------
| Q2에서 질의 에 대한 정보 요구 예상



In [17]:
ab_test(q1, q2, q3, thread_a="chat-003", thread_b="chat-004", thread_c="chat-005")



########## A/B MEMORY TEST START ##########

==================== A-1: seed memory (q1) (thread_id=chat-003) ====================
USER: 내 출생년도는 1994년이야. 지금은 계산하지 말고 이 정보만 기억해.

 RateLimit(429). retry in 1.5s ...

 RateLimit(429). retry in 3.0s ...

 RateLimit(429). retry in 4.5s ...

AI: 알겠습니다. 당신의 출생년도는 1994년입니다. 이 정보를 기억하겠습니다.

==================== B-1: q2 only (no memory) (thread_id=chat-004) ====================
USER: 그럼 내가 2026년에 몇 살인지 계산기를 꼭 사용해서 나이를 계산해서 알려줘
tool_calls: [{'name': 'calculator', 'args': {'expression': '2026-2023'}, 'id': 'call_uoBeNsrFAf1XEqiW0ynZPVcN', 'type': 'tool_call'}]

TOOL RESULT: calculator
3

AI: 2026년은 현재로부터 3년 후입니다. 따라서, 현재 나이에 3을 더하면 2026년에 몇 살인지 알 수 있습니다. 현재 나이를 알려주시면 2026년에 몇 살인지 계산해드릴 수 있습니다.

==================== C-1: q3 only (no memory) (thread_id=chat-005) ====================
USER: 올해는 무슨 해이며 나는 무슨 띠인지 알려줘

AI: 2023년은 음력으로 계묘년(癸卯年)이며, 이는 토끼띠에 해당합니다. 당신이 무슨 띠인지 알려면 태어난 해를 알아야 합니다. 태어난 해를 알려주시면 띠를 계산해드릴 수 있습니다.

########## A/B MEMORY TEST END #

# 3. Search-augmented ReAct (Self-ask with search)

*   질문 분해 중심
*   **RAG와 매우 유사**
*   사실상 Self-Ask = 초기 형태의 RAG



## ㄴ 3.1 Description

### ㄴ 3.1.1. Workflow



```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph / Agent Runtime 시작
  ↓
LLM이 질문을 "하위 질문(Sub-questions)"으로 분해
  ↓
각 하위 질문을 Search Tool로 순차 조회
  ↓
검색 결과(Observation)를 모아 중간 결론 생성
  ↓
필요하면 추가 하위 질문 생성 → 다시 Search
  ↓
모든 정보가 충분해지면 최종 답 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시
```



### ㄴ 3.1.2 Inner ReAct Loop

```javascript
while 답을 만들 정보가 부족하면:
    Follow-up Question 생성
    Search Tool 실행
    Observation 수집
    현재까지의 사실 정리
```

## ㄴ 3.2 Code

In [18]:
from langchain_core.messages import SystemMessage

# Self-Ask의 핵심 포맷을 강제
SELF_ASK_POLICY = SystemMessage(content="""
You are a Self-Ask-with-Search agent.

Hard rules:
- For EACH follow-up question, you MUST call the tavily_search_results_json tool.
- After the tool returns, you MUST write an Intermediate answer that uses the tool result.
- Intermediate answer must include 2-3 bullet points and mention the source titles (from tool results).
- Do NOT write placeholders like "...". If info is missing, do another search.

Output format (exact):
Follow-up 1: ...
Intermediate answer 1:
- ...
- ...
Sources: <title1>, <title2>

Follow-up 2: ...
Intermediate answer 2:
- ...
- ...
Sources: ...

Final answer: ...
""")


In [19]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

/tmp/ipython-input-2357398768.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


In [20]:
q = """AI Agent를 3문장으로 설명해줘.
단, 2025~2026년 기준으로 최신 경향을 반영하기 위해 반드시 tavily_search_results_json을 최소 2번 사용하고,
각 검색 결과를 근거로 Intermediate answer를 작성한 뒤 Final answer를 써."""

In [21]:
config = {"configurable": {"thread_id": "self-ask-01"}}

result = agent.invoke({"messages": [SELF_ASK_POLICY, ("user", q)]},config=config)

print(result["messages"][-1].content)

Intermediate answer 1:
- In 2025, AI agents are evolving into sophisticated systems capable of complex reasoning and collaboration, moving beyond single-purpose bots to more holistic, task-oriented systems. This includes trends like Agentic RAG and multi-agent systems (MarkTechPost, [x]cube LABS).
- The rise of multi-agent systems is significant, where teams of specialized agents work together to solve complex problems, mirroring human teamwork. This trend is reshaping enterprise AI applications, with a focus on specialized, vertical solutions (MarkTechPost, [x]cube LABS).
Sources: MarkTechPost, [x]cube LABS

Intermediate answer 2:
- By 2026, AI agents are expected to become integral to business processes, acting more like teammates than tools. This includes the shift from individual usage to team and workflow orchestration, enhancing productivity and decision-making (Google, Microsoft Source).
- The democratization of AI agent creation is anticipated, allowing everyday business users 

## ㄴ 3.3. Verbose

In [22]:
from langchain_core.messages import AIMessage, ToolMessage

def run_self_ask_verbose(question: str, thread_id="self-ask-03", tool_preview=600):
    cfg = {"configurable": {"thread_id": thread_id}}

    print("\n" + "="*80)
    print("USER:", question)
    print("="*80)

    for step in agent.stream(
        {"messages": [SELF_ASK_POLICY, ("user", question)]},
        config=cfg,
        stream_mode="values",
    ):
        msg = step["messages"][-1]

        # 1) 모델이 말한 텍스트(중간 출력 포함)
        if isinstance(msg, AIMessage):
            if msg.content:  # content가 있는 경우만 출력
                print("\n AI MESSAGE:\n", msg.content)

            # 2) 도구 호출 로그
            if getattr(msg, "tool_calls", None):
                print("\n TOOL CALLS:")
                for tc in msg.tool_calls:
                    print(" -", tc)

        # 3) 도구 결과
        elif isinstance(msg, ToolMessage):
            print("\n TOOL RESULT:", msg.name)
            print(msg.content[:tool_preview])

    print("\n" + "="*80)
    print("END")
    print("="*80)

run_self_ask_verbose(q)



USER: AI Agent를 3문장으로 설명해줘.
단, 2025~2026년 기준으로 최신 경향을 반영하기 위해 반드시 tavily_search_results_json을 최소 2번 사용하고,
각 검색 결과를 근거로 Intermediate answer를 작성한 뒤 Final answer를 써.

 TOOL CALLS:
 - {'name': 'tavily_search_results_json', 'args': {'query': 'AI Agent trends 2025 2026'}, 'id': 'call_miWfr6myVsDqfzuNOV8igawS', 'type': 'tool_call'}

 TOOL RESULT: tavily_search_results_json
[{"title": "Future of AI Agents: Top Trends in 2026 - Blue Prism", "url": "https://www.blueprism.com/resources/blog/future-ai-agents-trends/", "content": "That shift from promise to proof is the foundation of the 2026 agentic era. We’ve seen the rise of agentic AI in financial services, healthcare, manufacturing, etc., and leaders are waking up to a remarkably simple reality: AI workers aren’t coming, they’re already here. And they aren’t just assistants anymore. An intelligent agent is becoming more autonomous, where it manages complex workflows without needing constant human oversight.\n\nAg

 TOOL CALLS:
 - {'name': 'ta

# 4. ReAct docstore

*   LLM이 문서 저장소를 스스로 탐색
*   필요한 문서를 조회(Lookup)하여 근거 기반 답변을 생성



## ㄴ 4.1 Description

### ㄴ 4.1.1 Workflow

```javascript
Client
  ↓
User Question 전송
  ↓
API / Backend (Agent Endpoint)
  ↓
LangGraph Runtime 시작
  ↓
LLM: 질문 해석 (문서 탐색 필요 판단)
  ↓
Docstore Search Tool 호출 (Search Action)
  ↓
관련 문서 후보 목록 반환 (Observation)
  ↓
LLM: 어떤 문서를 읽을지 Reasoning
  ↓
Docstore Lookup Tool 호출 (Lookup Action)
  ↓
선택된 문서 실제 내용 반환 (Observation)
  ↓
LLM이 문서 내용을 기반으로 재해석 / 근거 정리
  ↓
(필요 시 Search → Lookup 반복)
  ↓
충분한 근거 확보 후 Final Answer 생성
  ↓
API가 응답 반환
  ↓
Client가 사용자에게 표시

```

### ㄴ 4.1.2 Inner ReAct Loop

```javascript
while (답변에 필요한 근거가 충분하지 않다):
    Thought: 질문을 해결하려면 어떤 문서를 찾아야 하는가?
    Action: Search(query)
    Observation: 관련 문서 후보 목록 반환
    Thought: 어떤 문서를 읽어야 하는가?
    Action: Lookup(document_id)
    Observation: 문서 내용 확보
    Thought:이 정보로 답변 가능한가?
            ├─ NO → 다른 문서 다시 Search
            └─ YES → 반복 종료
```

## ㄴ 4.2 Code

In [23]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools)

/tmp/ipython-input-2357398768.py:3: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


In [24]:
q = """
규칙:
- wikipedia 도구를 1회 호출해서 후보 제목을 찾는다.
- 그 다음 docstore_lookup 도구를 반드시 1회 이상 호출한다.
- docstore_lookup 결과를 근거로 최종 답변을 작성한다.

질문: 밍크 선인장(Mammillaria)은 어떤 식물인지 설명해줘.
"""

In [25]:

config = {"configurable": {"thread_id": "docstore-01"}}
result = agent.invoke({"messages": [("user", q)]},config=config)

print(result["messages"][-1].content)


밍크 선인장(Mammillaria)은 선인장과(Cactaceae)에서 가장 큰 속 중 하나로, 현재 200여 종과 변종이 알려져 있습니다. 대부분의 밍크 선인장은 멕시코가 원산지이며, 일부는 미국 남서부, 카리브해, 콜롬비아, 과테말라, 온두라스, 베네수엘라에서도 발견됩니다. 이 속은 일반적으로 "핀쿠션 선인장"으로 불리며, 이는 이 속과 밀접하게 관련된 Escobaria와도 관련이 있습니다.

밍크 선인장의 첫 번째 종은 1753년 칼 린네에 의해 Cactus mammillaris로 기술되었으며, 속명은 라틴어로 "젖꼭지"를 의미하는 mammilla에서 유래되었습니다. 이는 이 속의 독특한 특징 중 하나인 결절을 가리킵니다. 여러 종은 구형 선인장, 젖꼭지 선인장, 생일 케이크 선인장, 낚시바늘 선인장 또는 핀쿠션 선인장으로도 알려져 있습니다.


## ㄴ 4.3 Verbose

In [27]:
from langchain_core.messages import AIMessage, ToolMessage

def run_docstore_verbose(question: str, thread_id="docstore-debug"):
    cfg = {"configurable": {"thread_id": thread_id}}

    print("\n================ USER ================")
    print(question)

    for step in agent.stream(
        {"messages": [("user", question)]},
        config=cfg,
        stream_mode="values",   # 상태 변화 전부 받기
    ):
        msg = step["messages"][-1]

        # LLM이 생각한 내용 (Thought / Action 결정)
        if isinstance(msg, AIMessage):
            if msg.content:
                print("\n AI THOUGHT:")
                print(msg.content)

            if getattr(msg, "tool_calls", None):
                print("\n TOOL CALL:")
                print(msg.tool_calls)

        # Tool 실행 결과 (Observation)
        elif isinstance(msg, ToolMessage):
            print("\n TOOL RESULT:", msg.name)
            print(msg.content[:1000])  # 너무 길어서 제한

    print("\n================ END ================\n")

run_docstore_verbose(q)



================ USER ================

규칙:
- wikipedia 도구를 1회 호출해서 후보 제목을 찾는다.
- 그 다음 docstore_lookup 도구를 반드시 1회 이상 호출한다.
- docstore_lookup 결과를 근거로 최종 답변을 작성한다.

질문: 밍크 선인장(Mammillaria)은 어떤 식물인지 설명해줘.


 TOOL CALL:
[{'name': 'wikipedia', 'args': {'query': 'Mammillaria'}, 'id': 'call_ogxtJBav1SeJvLoBBHt6PIqA', 'type': 'tool_call'}]

 TOOL RESULT: wikipedia
Page: Mammillaria
Summary: Mammillaria is one of the largest genera in the cactus family (Cactaceae), with currently 200 known species and varieties recognized. Most of the mammillarias are native to Mexico, while some come from the Southwestern United States, the Caribbean, Colombia, Guatemala, Honduras and Venezuela. The common name "pincushion cactus" refers to this genus and the closely related Escobaria.
The first species was described by Carl Linnaeus as Cactus mammillaris in 1753, deriving its name from the Latin mammilla, "nipple", referring to the tubercles that are among the distinctive features of the genus. Numerous spec